In [0]:
#Run this command if you encounter typing_extensions error
pip install typing-extensions --upgrade

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# Installing the python snowflake connector
!pip install snowflake-connector-python

import snowflake.connector
import requests
import json

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d5e13118-ffbd-484b-997d-4b609ce6a582/bin/python -m pip install --upgrade pip' command.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-d5e13118-ffbd-484b-997d-4b609ce6a582/lib/python3.9/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (7.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [0]:
#function to connect to snowflake
def upload_data_to_snowflake(account, user, password, database, schema, warehouse, role):
    # Connect to Snowflake
    conn = snowflake.connector.connect(
        user=user,
        password=password,
        account=account,
        role=role
    )

    # Create a new database
    conn.cursor().execute(f'CREATE DATABASE IF NOT EXISTS {database}')
    # Switch to the new database
    conn.cursor().execute(f'USE DATABASE {database}')
    # Create a new schema
    conn.cursor().execute(f'CREATE SCHEMA IF NOT EXISTS {schema}')
    # Create a new warehouse
    conn.cursor().execute(f'CREATE WAREHOUSE IF NOT EXISTS {warehouse} WAREHOUSE_SIZE = XLARGE AUTO_SUSPEND = 300')
    # Switch to the new schema
    conn.cursor().execute(f'USE SCHEMA {schema}')

    # Define the table schema
    table_name = 'inspection'
    table_columns = []
    table_schema = ''
    
    #getting response and storing it in data variable 
    response = requests.get('https://data.cityofnewyork.us/resource/43nn-pn8j.json')
    
    #converting in JSON format
    data = json.loads(response.text)
    if len(data) > 0:
        for column in data[0]:
            if column == 'location_1':
                for sub_column in data[0][column]:
                    table_columns.append(sub_column)
                    table_schema += sub_column + ' STRING,'
            else:
                table_columns.append(column)
                table_schema += column + ' STRING,'
        table_schema = table_schema[:-1]  # Remove trailing comma

    # Create the new table
    if len(table_columns) > 0:
        conn.cursor().execute(f'CREATE TABLE IF NOT EXISTS {table_name} ({table_schema})')

        # Load data into the new table
        for row in data:
            values = []
            for column in table_columns:
                if column in row:
                    values.append(row[column])
                else:
                    values.append(None)
            query = f"INSERT INTO {table_name} ({','.join(table_columns)}) VALUES ({','.join(['%s']*len(values))})"
            conn.cursor().execute(query, values)

    # Close Snowflake connection
    conn.close()

# Calling the function
upload_data_to_snowflake('iv55434.central-india.azure', 'TYHAT81785', 'XXXX-PASSWORD-XXXX', 'Inspection', 'raw', 'nycwarehouse', 'ACCOUNTADMIN')